In [1]:
import sys
sys.path.append('..')
from ift6758.data import get_data
from ift6758.features import Game, ShotsEvent, Goal, ShotOnGoal, Player
import pandas as pd
from typing import List, Tuple, Dict, Any, Iterable

In [2]:
def extract_player_data(json_player_data:Dict[str, Any]) -> Dict[int, Player]:
    players_dict = {}
    for player in json_player_data:
        players_dict[player["playerId"]] = Player(player)
    return players_dict
        
def import_game_stats(game_id:int) -> Tuple[List[ShotsEvent], Dict[int, Player]]:
    game_data = get_data.retrieve_game_data(game_id, verbose=True)
    players_dict = extract_player_data(game_data["rosterSpots"])
    events = game_data["plays"]
    game : Game = Game.get_game(game_id)
    print("printing game :", game)
    for event in events:
        try:
            if event["typeDescKey"] == "goal":
                e : Goal = Goal(game, event)
            elif event["typeDescKey"] == "shot-on-goal":
                e : ShotOnGoal = ShotOnGoal(game, event)
        except KeyError:
            continue
    return ShotsEvent.get_events_by_game(game_id), players_dict

def game_stats_to_table(game_id : int, df : List[Dict[str, Any]] = []) -> List[Dict[str, Any]]:
    event_list, players_dict = import_game_stats(game_id)
    game: Game = Game.get_game(game_id)
    game_dict = game.to_dict()
    for event in event_list:
        event_json = event.to_dict()
        event_json.update(game_dict)
        event_json['type'] = event.get_event_type()
        df.append(event_json)
    return df

def games_to_table(game_ids : Iterable) -> pd.DataFrame:
    df = []
    for game_id in game_ids:
        if len(str(game_id)) == 4:
            games = get_data.regular_season_game_id_generator(game_id)
            for game in games:
                print("game", game)
                df = game_stats_to_table(game, df)
        else:
            df = game_stats_to_table(game_id, df)
    return pd.DataFrame(df)


In [ ]:
df = pd.DataFrame(games_to_table([2017, 2018]))
print(df)

In [ ]:
df.to_csv("../ift6758/data/preprocessed_data/preprocessed_game_data.csv", index=False)

In [ ]:
unique_event_types = df['type'].unique()
unique_event_types

In [ ]:
events_with_no_zone_code = df[df['zoneCode'].isna()]
events_with_no_zone_code

In [ ]:
events_with_no_shot_type = df[df['shotType'].isna()]
events_with_no_shot_type

In [ ]:
events_with_no_goalie = df[df['goalieInNetId'].isna()]
events_with_no_goalie # net empty of no goalie ?

In [ ]:
# errors ?
events_with_no_x_coord = df[df['xCoord'].isna()]
events_with_no_x_coord

In [ ]:
events_with_no_y_coord = df[df['yCoord'].isna()]
events_with_no_y_coord